In [1]:
import pandas as pd
import numpy as np
import os
import string
import glob
import random
import cmath
import math

In [2]:
Dimen=1000
Ngram=3

In [3]:
#########Loading data##################

LangLabels = ['bul', 'ces', 'dan', 'nld', 'deu', 'eng', 'est', 'fin', 'fra', 'ell', 'hun', 'ita',
              'lav', 'lit', 'pol', 'por', 'ron', 'slk', 'slv', 'spa', 'swe']
Traindata=[]
Trainlabel=[]
for i, Label in enumerate(LangLabels):
    FilePath = os.path.join('language','training_texts',"{}.txt".format(Label))
    with open(FilePath,'r', encoding='utf-8') as f:
        inputseq=f.read()
    senlist=inputseq.split('\n')
    for data in senlist:
        Traindata.append(data)
        Trainlabel.append(i)
print("{} sentences have been loaded for train".format(str(len(Traindata))))

210080 sentences have been loaded for train


In [4]:
ShortLangLabels = ['bg', 'cs', 'da', 'nl', 'de', 'en', 'et', 'fi', 'fr', 'el', 'hu', 'it', 'lv', 'lt', 'pl', 'pt', 'ro', 'sk', 'sl', 'es', 'sv']
Testdata=[]
Testlabel=[]
mp = dict()
for i,key in enumerate(ShortLangLabels):
    mp[key] = i
TestFilePathList = glob.glob(os.path.join("language", "testing_texts", "*.txt"))
for i, TestFilePath in enumerate(TestFilePathList):
    (_, TestFileName) = os.path.split(TestFilePath)
    if not mp.__contains__(TestFileName[0:2]):
        continue
    Label = mp[TestFileName[0:2]]
    with open(TestFilePath,'r',encoding='utf-8') as f:
        InputSequence = f.read()
    Testdata.append(InputSequence[:-1]) 
    Testlabel.append(Label)
print("{} sentences have been loaded for test.".format(str(len(Testdata))))

21000 sentences have been loaded for test.


In [27]:
#Polar
K=21
IM = dict() 
AM = dict()
IAM = dict()
LangLabels = []
for key in range(K):
    AM[key] = np.zeros(Dimen)
    LangLabels.append(key)
    
def GenerateHV():
    real_part = np.random.uniform(low=-1, high=1, size=Dimen)
    complex_part = np.random.uniform(low=-1, high=1, size=Dimen)
    RandomHV = real_part + 1j * complex_part
    RandomHV = [cmath.phase(i) for i in RandomHV]
    return RandomHV

def CalculateNormalSimilarity(vector1, vector2):
     return np.dot(vector1, vector2)

def LookupIM(Key):
    if not IM.__contains__(Key):
        IM[Key] = GenerateHV();
    return IM[Key]

def NGramEncoding(InputSequence):
    block = np.zeros([Ngram, Dimen])
    sumHV = np.zeros(Dimen)
    for i, Key in enumerate(InputSequence):
        block = np.roll(block, (1, 1), (0, 1))
        block[0] = LookupIM(Key)
        if i >= Ngram - 1:
            NGramHV = block[0]
            for j in range(1, Ngram):
                NGramHV = NGramHV * block[j]
            sumHV += np.multiply(2,NGramHV)    #//////////////////////////
    return sumHV

def Normalization(vector):
    vector /= np.linalg.norm(vector)   #//////////////////////
    return vector

length = len(Traindata)
for i in range(length):
    Data = Traindata[i]
    Label = Trainlabel[i]
    tmp = NGramEncoding(Data)
    AM[Label] = AM[Label] + tmp
for Label in LangLabels:
    IAM[Label] = AM[Label]
    AM[Label] = Normalization(AM[Label])


C:\Users\mdasi\AppData\Local\Temp\ipykernel_6060\1174679383.py:40: RuntimeWarning: invalid value encountered in true_divide
  vector /= np.linalg.norm(vector)   #//////////////////////


In [28]:
length = len(Testdata)
predict = []
for i in range(length):
    Data = Testdata[i]
    TestHV = Normalization(NGramEncoding(Data))
    maxAngle = -1
    predicLang = -1
    for j,LangLabel in enumerate(LangLabels):
        angle = CalculateNormalSimilarity(AM[LangLabel], TestHV)
        if angle > maxAngle:
            maxAngle = angle
            predicLang = LangLabel
    predict.append(predicLang)

In [29]:
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
print("Accuracy score: ",accuracy_score(Testlabel,predict))

Accuracy score:  0.0
